In [1]:
from tensorflow.keras.applications import VGG16

In [2]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
import matplotlib.pyplot as plt

In [5]:
import numpy as np

import os

In [6]:
import json
BASE_WORK_PATH = "/home/myn1szh/AI_2021/sign-plate-detection/"
with open(BASE_WORK_PATH + "sign-plate-detection_train.json") as jf:
    train_ds = json.load(jf)

In [11]:
print("[INFO] loading dataset...")

data = []
targets = []
filenames = []
# loop over the row
for i in range(len(train_ds["annotations"])):
    image_name = train_ds["images"][train_ds["annotations"][i]["image_id"]]["file_name"]
    filename = BASE_WORK_PATH + "images/" + image_name
    h = 200
    w = 200
    startX = train_ds["annotations"][i]["bbox"][0]
    startY = train_ds["annotations"][i]["bbox"][1]
    endX = startX + train_ds["annotations"][i]["bbox"][2]
    endY = startY + train_ds["annotations"][i]["bbox"][3]
    # scale the bounding box coordinates relative to the spatial
    # dimensions of the input image
    startX = float(startX)/w
    startY = float(startY) / h
    endX = float(endX) / w
    endY = float(endY) / h
    # load the image and preprocess it
    image = load_img(filename, target_size=(200, 200))
    image = img_to_array(image)
    # update our list of data, targets, and filenames
    data.append(image)
    targets.append((startX, startY, endX, endY))
    filenames.append(filename)
    # convert the data and targets to NumPy arrays, scaling the input
# pixel intensities from the range [0, 255] to [0, 1]
data = np.array(data, dtype="float32") / 255.0
targets = np.array(targets, dtype="float32")
# partition the data into training and testing splits using 90% of
# the data for training and the remaining 10% for testing
split = train_test_split(data, targets, filenames, test_size=0.10,random_state=42)
# unpack the data split
(trainImages, testImages) = split[:2]
(trainTargets, testTargets) = split[2:4]
(trainFilenames, testFilenames) = split[4:]
# write the testing filenames to disk so that we can use then
# when evaluating/testing our bounding box regressor
print("[INFO] saving testing filenames...")


[INFO] loading dataset...
[INFO] saving testing filenames...


In [12]:
INIT_LR = 1e-4
NUM_EPOCHS = 25
BATCH_SIZE = 32

# load the VGG16 network, ensuring the head FC layers are left off
#vgg = VGG16(weights="imagenet", include_top=False,input_tensor=Input(shape=(200, 200, 3)))
vgg = VGG16(weights="vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5", 
            include_top=False,input_tensor=Input(shape=(200, 200, 3)))


In [13]:
# freeze all VGG layers so they will *not* be updated during the
# training process
vgg.trainable = False
# flatten the max-pooling output of VGG
flatten = vgg.output
flatten = Flatten()(flatten)
# construct a fully-connected layer header to output the predicted
# bounding box coordinates
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bbox_head = Dense(4, activation="sigmoid")(bboxHead)
bbox_probs = Dense(1, activation="softmax")(bboxHead)
# construct the model we will fine-tune for bounding box regression
model = Model(inputs=vgg.input, outputs=[bbox_probs,bbox_head])

In [14]:
# initialize the optimizer, compile the model, and show the model
# summary
INIT_LR = 1e-4
opt = Adam(lr=INIT_LR)
model.compile(loss="mse", optimizer=opt)
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 200, 200, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 200, 200, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 100, 100, 64) 0           block1_conv2[0][0]               
____________________________________________________________________________________________

In [29]:
# train the network for bounding box regression
print("[INFO] training bounding box regressor...")
"""
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto
config.gpu_options.allow_growth = True
seesion=InteractiveSession(config=config)
"""

INIT_LR = 1e-4
NUM_EPOCHS = 5
BATCH_SIZE = 4
H = model.fit(
    trainImages, trainTargets,
    validation_data=(testImages, testTargets),
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    verbose=1)



[INFO] training bounding box regressor...


AttributeError: 'google.protobuf.pyext._message.FieldProperty' object has no attribute 'allow_growth'

In [ ]:
# serialize the model to disk
print("[INFO] saving object detector model...")
model.save("test.h5", save_format="h5")
# plot the model training history
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.title("Bounding Box Regression Loss on Training Set")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.savefig("plot_path")

In [ ]:
# load our trained bounding box regressor from disk
print("[INFO] loading object detector...")
model = load_model(config.MODEL_PATH)
# loop over the images that we'll be testing using our bounding box
# regression model
for imagePath in imagePaths:
	# load the input image (in Keras format) from disk and preprocess
	# it, scaling the pixel intensities to the range [0, 1]
	image = load_img(imagePath, target_size=(200, 200))
	image = img_to_array(image) / 255.0
	image = np.expand_dims(image, axis=0)
    	# make bounding box predictions on the input image
	preds = model.predict(image)[0]
	(probs, startX, startY, endX, endY) = preds
	# load the input image (in OpenCV format), resize it such that it
	# fits on our screen, and grab its dimensions
	image = cv2.imread(imagePath)
	image = imutils.resize(image, width=600)
	(h, w) = image.shape[:2]
	# scale the predicted bounding box coordinates based on the image
	# dimensions
	startX = int(startX * w)
	startY = int(startY * h)
	endX = int(endX * w)
	endY = int(endY * h)
	# draw the predicted bounding box on the image
	cv2.rectangle(image, (startX, startY), (endX, endY),
		(0, 255, 0), 2)
	# show the output image
	cv2.imshow("Output", image)
